In [1]:
)load isolate
isolate.Config 'listen' 1

/opt/mdyalog/19.0/64/unicode/ws/isolate.dws ⍝ saved Wed Sep 4 22:43:51 2024
 
 For information on how to use Futures and Isolates, 
 see the manual "Parallel Language Features", 
 which can be found in the Dyalog Documentation Centre 
 at http://docs.dyalog.com

0

In [2]:
SNRs ← (¯35 + (⍳50)÷2)
iso ← ø¨(≢SNRs)⍴⊂''
iso.SNR ← SNRs

In [3]:
'InitCauseway' 'View' ⎕CY'sharpplot'

In [4]:
]dinput
plot ← {
    do  ← {⍎'⍺⍺ ⍵ ⋄ ⍵' ⋄ ⍺⍺}
    _   ← InitCauseway do ⍬
    sp  ← ⎕NEW Causeway.SharpPlot
    ⍝sp.YAxisStyle ← Causeway.YAxisStyles.LogScale + Causeway.YAxisStyles.ClampToAxis
    sp.Heading ← (2⊃⍺)
    sp.XCaptions ← (3⊃⍺)
    sp.YCaptions ← (4⊃⍺)
    ⍝sp.LineGraphStyle ← Causeway.LineGraphStyles.GridLines
    _   ← sp.DrawLineGraph do ⍵
    sp.SavePdf (1⊃⍺) Causeway.PdfMode.ShowOutlines
    ⍝svg ← sp.RenderSvg Causeway.SvgMode.FixedAspect
    ⍝_   ← 'Reaction Times' (3500⌶) svg
}

In [5]:
]dinput
fft ← {
    k    ← 2÷⍨≢⍵
    1>k : ⍵
    0≠1 | 2⍟≢⍵: 'la taille doit etre un mult de 2'
    odd  ← ∇((≢⍵)⍴1 0)/⍵
    even ← ∇((≢⍵)⍴0 1)/⍵
    exp  ← *(0J¯2×(○1)×(¯1+⍳k)÷≢⍵)
    T    ← even × exp
    (odd+T), (odd-T)
}

In [6]:
iso.SF ← SF ← 8                 ⍝ spread factor
iso.ns ← ns ← 2*12              ⍝ nombre de samples at least 19
iso.B  ← B  ← 125e3             ⍝ bandwitdh
iso.f0 ← f0 ← 0
iso.T  ← T  ← 1 ÷ B             ⍝ periode
iso.Ts ← Ts ← T × 2*SF          ⍝ periode de symbole
iso.t  ← ⊂(¯1+⍳ns)×Ts÷ns   ⍝ variable du temps
t  ← (¯1+⍳ns)×Ts÷ns
iso.Ps ← Ps ← 1

In [7]:
Re  ← { (⍵ + +⍵) ÷ 2 }

In [8]:
]dinput
chirp ← {
    s   ← ⍵
    f   ← ((2*SF) | (t÷T) + s) × (B÷(2*SF))
    phi ← ○2 × +\ (f0+f) × Ts÷ns
    *(0J1 × phi)
}

In [9]:
]dinput
chirp_string ← {
    w ← ⎕UCS ⍵
    ∊chirp¨w
}

In [19]:
⍝ https://github.com/Dyalog/pynapl.git
]load ../pynapl/pynapl/Py
py ← ⎕NEW Py.Py
plt ← py.Import 'matplotlib.pyplot'

#.Py

In [10]:
]dinput
gauss ← {
   depth ← 2*32                             ⍝ profondeure de aletoirement
   (x y) ← ⊂[1+⍳⍴,⍵](?(2,⍵)⍴ depth) ÷ depth ⍝ deux variables aleatoires dans ]0;1]
   ((¯2×⍟x) * 0.5) × *(0J1 × ○2 × y)        ⍝ methode de Box-Muller
}

In [11]:
noise ← { ((Ps × 10*SNR÷¯10) * 0.5) × gauss ≢⍵ }

In [21]:
chop ← { s ← ≢⍵ ⋄ ws ← s÷16 ⋄ sk ← 15 ⋄ ( (1+s-ws) ⍴ ((sk⍴0), 1))/ (ws ,/ ⍵) }

In [13]:
spectrogram ← { N ← 2 ⋄ {((≢⍵)÷N)↑⍵} |fft ¨ chop ⍵ (,⍣N)⍬ }

In [32]:
sym ← 125
c   ← chirp sym
SNR ← 0
r   ← c + noise c
('c_s.pdf' 'signal chirp dans le domain temporiel' 'c' 't') plot (Re c) t
('noise.pdf' 'bruit gaussian additive blanc' 'n' 't') plot (Re noise c) t
('signal0.pdf' 'signal bruite a SNR = 0' 'n' 't') plot (Re r) t
SNR ← ¯10
r   ← c + noise c
('signal10.pdf' 'signal bruite a SNR = 10' 'n' 't') plot (Re r) t

In [35]:
plt.imshow ⊂↓⌽⍉↑spectrogram c
plt.show ⍬
plt.imshow ⊂↓⌽⍉↑spectrogram r
plt.show ⍬
plt.imshow ⊂↓⌽⍉↑spectrogram dechirp ⍬
plt.show ⍬
plt.imshow ⊂↓⌽⍉↑spectrogram (r×dechirp ⍬)
plt.show ⍬

In [ ]:
1

In [16]:
dechirp ← { s ← {⍵ ≡ ⍬: 0 ⋄ ⍵}⍵ ⋄ ⊖+chirp s }

In [ ]:

plt.imshow ⊂ spectrogram (c×dechirp ⍬)
plt.show ⍬

In [ ]:
c   ← chirp sym
SNR ← ¯3
r   ← c + noise c
('noisyscalar.pdf' 'produit scalaire du signal bruite' '<r,s>' 's') plot |fft r×dechirp ⍬
('perfectscalar.pdf' 'produit scalaire du signal parfait' '<r,s>' 's') plot |fft c×dechirp ⍬

In [18]:
s   ← 128
c   ← chirp s
SNR ← ¯3
r   ← c + noise c
('noisyfft.pdf' 'transforme du Fourier du signal bruite' 'fft(r,s)' 's') plot | fft r×dechirp ⍬
('perfectfft.pdf' 'transforme du Fourier du signal parfait' 'fft(r,s)' 's') plot | fft c×dechirp ⍬

In [ ]:
demodulate ← { ¯1 + ⊃⍒|fft ⍵×dechirp ⍬ }

In [ ]:
]dinput
check ← {
    c   ← chirp ⍵
    r   ← c + noise c
    ⍵ = demodulate r
}

In [ ]:
simulate ← { r ← ⍵ ÷⍨ ⍵ - +/check¨?⍵⍴¯1+2*SF ⋄ r = 0 : 100÷⍵ ⋄ r×100 }

In [ ]:
varieSF ← { SF ⊢← ⍵ ⋄ ns ⊢← 2*SF ⋄ Ts ⊢← T × 2*SF ⋄ t ⊢← (¯1+⍳ns)×Ts÷ns ⋄ simulate 1000 }

In [ ]:
_←iso.⎕FX ⊂⎕CR 'fft'
_←iso.⎕FX ⊂⎕CR 'check'
_←iso.⎕FX ⊂⎕CR 'chirp'
_←iso.⎕FX ⊂⎕CR 'noise'
_←iso.⎕FX ⊂⎕CR 'demodulate'
_←iso.⎕FX ⊂⎕CR 'gauss'
_←iso.⎕FX ⊂⎕CR 'dechirp'
_←iso.⎕FX ⊂⎕CR 'simulate'
_←iso.⎕FX ⊂⎕CR 'varieSF'

In [ ]:
SFhh  ← iso.(varieSF¨7+⍳9)

In [ ]:
plot (↓⍉↑SFhh) SNRs

In [ ]:
cfar ← { N ← 10 ⋄ cut ← ((-N)↓N↓⍵) ⋄ cut > (N÷2) ÷⍨ cut -⍨ +/¨(N+1+N) ,/ ⍵ }